In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import argparse
import logging
import sys
import time

from tf_pose import common
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import matplotlib.pyplot as plt
import glob
import os
import random

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))

[2018-07-30 19:44:02,668] [TfPoseEstimator] [INFO] loading graph from /Users/rui/vislab-workspace/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)


In [95]:
# estimate human poses from a single image !
with open('/Users/rui/Downloads/FDDB-folds/FDDB.txt') as f:
    files_grabbed = [line.rstrip('\n') for line in f]

i = random.randint(0,len(files_grabbed))
path = 'originalPics/'+files_grabbed[i]+'.jpg'

In [ ]:
with open('tf_pose_estimation.txt',"w") as f:
    for image_name in log_progress(files_grabbed):
        path = 'originalPics/'+image_name+'.jpg'
        image = common.read_imgfile(path, None, None)
        humans = e.inference(image, resize_to_default=(432 > 0 and 368 > 0), 
                                 upsample_size=4)
        f.write(image_name + '\n')
        img_h, img_w = image.shape[:2]
        faces = [human.get_face_box(img_w,img_h) for human in humans]
        faces = [face for face in faces if face is not None]
        f.write(str(len(faces)) + '\n')
        for face in faces:
            f.write('{} {} {} {} {}\n'.format(face['x'],face['y'],face['w'],face['h'],face['confidence']) )
    

VBox(children=(HTML(value=''), IntProgress(value=0, max=2845)))

In [84]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=True)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))